# **Prepare Data**

In [1]:
pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 8.4 MB/s eta 0:00:00
  

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import timm
import cv2
import albumentations as A
from glob import glob
import os
from autogluon.multimodal import MultiModalPredictor
from tqdm.notebook import tqdm

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [3]:
train_df = pd.read_csv('/kaggle/input/dog-breed-identification/labels.csv')
sub_df = pd.read_csv('/kaggle/input/dog-breed-identification/sample_submission.csv')
test_df = pd.DataFrame({
    'id':sub_df['id'],
    'path':sub_df['id'].astype(str).apply(lambda x:'/kaggle/input/dog-breed-identification/test/' + x + '.jpg')
})

In [4]:
train_df['path'] = train_df['id'].astype(str).apply(lambda x:'/kaggle/input/dog-breed-identification/train/' + x + '.jpg')

# **Explore Data**

In [5]:
train_df.head()

,id,breed,path
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull,/kaggle/input/dog-breed-identification/train/0...
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo,/kaggle/input/dog-breed-identification/train/0...
2,001cdf01b096e06d78e9e5112d419397,pekinese,/kaggle/input/dog-breed-identification/train/0...
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick,/kaggle/input/dog-breed-identification/train/0...
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever,/kaggle/input/dog-breed-identification/train/0...


In [6]:
test_df.head()

,id,path
0,000621fb3cbb32d8935728e48679680e,/kaggle/input/dog-breed-identification/test/00...
1,00102ee9d8eb90812350685311fe5890,/kaggle/input/dog-breed-identification/test/00...
2,0012a730dfa437f5f3613fb75efcd4ce,/kaggle/input/dog-breed-identification/test/00...
3,001510bc8570bbeee98c8d80c8a95ec1,/kaggle/input/dog-breed-identification/test/00...
4,001a5f3114548acdefa3d4da05474c2e,/kaggle/input/dog-breed-identification/test/00...


In [7]:
train_df.shape,test_df.shape

((10222, 3), (10357, 2))

In [ ]:
uni_label = sorted(train_df['breed'].unique())
num = len(uni_label)
col = 4
row = int(num/col)
plt.figure(figsize=(20,5*row))
for i,label in enumerate(uni_label):
    img_row = train_df[train_df['breed'] == label].iloc[0]
    img_path = img_row['path']
    img = mpimg.imread(img_path)
    plt.subplot(row,col,i+1)
    plt.imshow(img)
    plt.title(label)
plt.tight_layout()
plt.show

# **Augmentation**

In [9]:
os.makedirs("train_aug", exist_ok=True)
aug = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=30, p=0.7),
    A.RandomBrightnessContrast(p=0.2)
])

In [10]:
new_rows = []
for idx,row in tqdm(train_df.iterrows(),total=len(train_df)):
    img_path = row['path']
    image = cv2.imread(img_path)
    for i in range(2):
        augment = aug(image=image)['image']
        new_filename = f"train_aug/{row['id']}_aug_{i}.jpg"
        cv2.imwrite(new_filename,augment)
        new_row = row.copy()
        new_row['path'] = new_filename
        new_row['id'] = f"{row['id']}_aug_{i}"
        new_rows.append(new_row)

  0%|          | 0/10222 [00:00<?, ?it/s]

In [11]:
aug_df = pd.DataFrame(new_rows)
train_aug_df = pd.concat([train_df,aug_df],axis = 0).reset_index(drop=True)
print(f"Original shape: {train_df.shape}")
print(f"Augmented shape: {train_aug_df.shape}")

Original shape: (10222, 3)
Augmented shape: (30666, 3)


# **Model**

In [12]:
label = 'breed'
metric = 'log_loss'

predictor = MultiModalPredictor(label = label,eval_metric = metric)
predictor.fit(
    time_limit = 3600,
    presets = 'best_quality',
    train_data = train_aug_df
)

No path specified. Models will be saved in: "AutogluonModels/ag-20260123_061132"
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Sep 27 10:16:09 UTC 2025
CPU Count:          4
Pytorch Version:    2.8.0+cu126
CUDA Version:       CUDA is not available
GPU Memory:         
Total GPU Memory:   Free: 0.00 GB, Allocated: 0.00 GB, Total: 0.00 GB
GPU Count:          0
Memory Avail:       29.15 GB / 31.35 GB (93.0%)
Disk Space Avail:   17.98 GB / 19.52 GB (92.1%)
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	First 10 (of 120) unique label values:  ['boston_bull', 'dingo', 'pekinese', 'bluetick', 'golden_retriever', 'bedlington_terrier', 'borzoi', 'basenji', 'scottish_deerhound', 'shetland_sheepdog']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter d

model.safetensors:   0%|          | 0.00/788M [00:00<?, ?B/s]

GPU Count: 0
GPU Count to be Used: 0

/usr/local/lib/python3.12/dist-packages/autogluon/multimodal/utils/precision.py:71: UserWarning: Only CPU is detected in the instance. This may result in slow speed for MultiModalPredictor. Consider using an instance with GPU support.
  warnings.warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name              | Type                            | Params | Mode 
------------------------------------------------------------------------------
0 | model             | TimmAutoModelForImagePrediction | 195 M  | train
1 | validation_metric | MeanMetric                      | 0      | train
2 | loss_func         | CrossEntropyLoss                | 0      | train
------------------------------------------------------------------------------
195 M     Trainable params
0         Non-trainable params
195 M     Total params
780.720   Total estimated model params size (MB)
467       Modules in train mode
0         Modules in e

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Time limit reached. Elapsed time is 1:00:00. Signaling Trainer to stop.


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 0, global step 11: 'val_log_loss' reached 4.82168 (best 4.82168), saving model to '/kaggle/working/AutogluonModels/ag-20260123_061132/epoch=0-step=11.ckpt' as top 3
AutoMM has created your model. 🎉🎉🎉

To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/kaggle/working/AutogluonModels/ag-20260123_061132")
    ```

If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub (https://github.com/autogluon/autogluon/issues).




# **Evaluate**

In [13]:
predictor.evaluate(train_df)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


Predicting: |          | 0/? [00:00<?, ?it/s]

{'log_loss': 4.819211086630077}

# **Submission**

In [14]:
pred = predictor.predict_proba(test_df)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


Predicting: |          | 0/? [00:00<?, ?it/s]

In [15]:
sub_df = pd.DataFrame()
sub_df['id'] = test_df['id']
sub_df = pd.concat([sub_df, pred], axis=1)
sub_df = sub_df[sub_df.columns]

In [16]:
sub_df.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.012026,0.008564,0.009529,0.007138,0.008205,0.008885,0.008425,0.008684,0.008274,...,0.012211,0.006690,0.009442,0.007416,0.007638,0.010094,0.006527,0.009348,0.009082,0.006661
1,00102ee9d8eb90812350685311fe5890,0.007612,0.008581,0.010021,0.006163,0.008403,0.006935,0.007651,0.006462,0.008886,...,0.011995,0.007712,0.010010,0.009679,0.007163,0.010575,0.005872,0.006966,0.008273,0.007289
2,0012a730dfa437f5f3613fb75efcd4ce,0.006086,0.006810,0.006562,0.006188,0.006285,0.010020,0.007260,0.009422,0.005882,...,0.009710,0.005368,0.007323,0.006919,0.008408,0.007334,0.007332,0.008666,0.008543,0.010003
3,001510bc8570bbeee98c8d80c8a95ec1,0.007035,0.010815,0.006130,0.006336,0.008789,0.005941,0.014292,0.006079,0.013881,...,0.017302,0.014276,0.008899,0.007209,0.006200,0.011443,0.009799,0.009060,0.007465,0.003454
4,001a5f3114548acdefa3d4da05474c2e,0.011767,0.006909,0.010494,0.006764,0.006586,0.007693,0.011532,0.008420,0.007912,...,0.008632,0.007188,0.008279,0.007607,0.008475,0.008744,0.005186,0.008509,0.006833,0.005672


In [17]:
sub_df.to_csv('submission.csv', index=False)